### Notebook for noodling with screenshot reading algorithm.

Output should consist of a dataframe that has text, placement, prominence, maybe color?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import ipdb
import pickle
import pytesseract
import string
import sys
import time
import math
import csv
import re
import cv2